In [10]:
import pandas as pd
import numpy as np
import subscribers_module as sub

import sys
sys.path.append('../../zoho_data_app/')

# importing
import main_module as m

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
list = sub.load_lists_id() # load list id

#load data mailwizz

maillwizz_cols = ['subscriber_uid',	
'EMAIL',
'FNAME',	
'LNAME',	
'ALAMAT',	
'STATUS']

zoho_df_cols = ['Email',
'First name',	
'Last name',	
'Address',	
'Status']


In [12]:
load_maillwizz_df = sub.load_subs(list['Tiket Konser'])
load_maillwizz_df = sub.reindex_cols(load_maillwizz_df, maillwizz_cols)
load_maillwizz_df

,subscriber_uid,EMAIL,FNAME,LNAME,ALAMAT,STATUS
0,kz492769e9753,toni6@gmail.com,Goku,Saiyan,Solo,Maried
1,gt985nps3k2d5,toni5@gmail.com,Yoimiya,Berthold,Gorontalo,Maried
2,zs591efhjm0fd,toni4@gmail.com,Boruto,Hercules,Medan,Single
3,zd239c94r41b9,toni3@gmail.com,Nahida,Stark,Jogja,Single
4,ve506lssv74e3,toni1@gmail.com,Toni,Ackerman,Denpasar,Single


In [13]:
zoho_df_dict = {'Email':['toni1@gmail.com', 'toni3@gmail.com', 'toni4@gmail.com', 'toni5@gmail.com', 'toni6@gmail.com'], 
            'First name':['Toni', 'Nahida', 'Boruto', 'Yoimiya', 'Goku'],
            'Last name':['Ackerman', 'Stark', 'Hercules', 'Berthold', 'Saiyan'],
            'Address':['Denpasar', 'Jogja', 'Medan', 'Gorontalo', 'Solo'],
            'Status':['Single', 'Single', 'Single', 'Maried', 'Maried']
        }
zoho_df= pd.DataFrame.from_dict(zoho_df_dict)
zoho_df = sub.reindex_cols(zoho_df, zoho_df_cols)
zoho_df

,Email,First name,Last name,Address,Status
0,toni1@gmail.com,Toni,Ackerman,Denpasar,Single
1,toni3@gmail.com,Nahida,Stark,Jogja,Single
2,toni4@gmail.com,Boruto,Hercules,Medan,Single
3,toni5@gmail.com,Yoimiya,Berthold,Gorontalo,Maried
4,toni6@gmail.com,Goku,Saiyan,Solo,Maried


In [14]:
input_new_subs_zoho = sub.input_new_subs_from_zoho(zoho_df, 'Email', load_maillwizz_df, 'EMAIL')
deleted_mailwizz_df = sub.delete_maillwizz_not_exist_zoho(load_maillwizz_df, 'EMAIL', zoho_df, 'Email')

In [15]:
load_maillwizz_df = sub.load_subs(list['Tiket Konser'])

diff_field_email = sub.sync_maillwizz_zoho_field(zoho_df, 'Email', load_maillwizz_df, 'EMAIL')
diff_field_email.loc[
    (diff_field_email['FNAME'] != diff_field_email['First name']) |
    (diff_field_email['LNAME'] != diff_field_email['Last name']) |
    (diff_field_email['ALAMAT'] != diff_field_email['Address']) |
    (diff_field_email['STATUS'] != diff_field_email['Status'])
    ]

,subscriber_uid,EMAIL,ALAMAT,STATUS,FNAME,LNAME,status,source,ip_address,date_added,Email,First name,Last name,Address,Status


In [16]:
updated_subs = diff_field_email.loc[
    (diff_field_email['FNAME'] != diff_field_email['First name']) |
    (diff_field_email['LNAME'] != diff_field_email['Last name']) |
    (diff_field_email['ALAMAT'] != diff_field_email['Address']) |
    (diff_field_email['STATUS'] != diff_field_email['Status'])
    ].copy()

updated_subs['ALAMAT'] = updated_subs['Address']
updated_subs['STATUS'] = updated_subs['Status']
updated_subs['FNAME'] = updated_subs['First name']
updated_subs['LNAME'] = updated_subs['Last name']

updated_subs.reset_index(drop=True, inplace=True)

updated_subs


,subscriber_uid,EMAIL,ALAMAT,STATUS,FNAME,LNAME,status,source,ip_address,date_added,Email,First name,Last name,Address,Status


In [8]:
updated_subs = sub.reindex_cols(updated_subs, maillwizz_cols)
updated_subs

# bulk update subs
sub.update_data_subs(list['Tiket Konser'], updated_subs)

'success update'

In [236]:
#load_maillwizz_df.loc[load_maillwizz_df['EMAIL'].str.contains('toni')]